INPUTS
- `sset.PATH_CIAM_COASTLINES`
- `sset.PATH_GADM_ADM1_VORONOI`

OUTPUTS
- `sset.PATH_CIAM_COASTLINES_BY_ISO`

In [ ]:
from sliiders import settings as sset
from sliiders import spatial

spatial.filter_spatial_warnings()

In [ ]:
coastlines = spatial.get_coastlines_by_iso(
    sset.PATH_CIAM_COASTLINES, sset.PATH_GADM_ADM1_VORONOI
)

### Save

In [ ]:
coastlines.to_parquet(sset.PATH_CIAM_COASTLINES_BY_ISO, index=False)